# `FileStore` examples

## CSV example
Here we create an example CSV file, post it to Coop using `FileStore`, and then retrieve it and use it to construct a `ScenarioList` and an `AgentList`.

To create a CSV file (this step can be skipped and your own file replaced in the following steps):

In [1]:
data = [
    ['Age', 'City', 'Occupation'],
    [25, 'New York', 'Software Engineer'],
    [30, 'San Francisco', 'Teacher'],
    [35, 'Chicago', 'Doctor'],
    [28, 'Boston', 'Data Scientist'],
    [45, 'Seattle', 'Architect']
]

# Writing to CSV file
with open('data.csv', 'w') as file:
    for row in data:
        line = ','.join(str(item) for item in row)
        file.write(line + '\n')

Posting to Coop:

In [2]:
from edsl import FileStore

fs = FileStore("data.csv")
csv_info = fs.push(description = "My example CSV file", alias = "my-example-csv-file", visibility = "public")
csv_info # display the URL and Coop uuid of the stored file for retrieving it later

{'description': 'My example CSV file',
 'object_type': 'scenario',
 'url': 'https://www.expectedparrot.com/content/a2eeb5a1-1f43-4380-8aa7-6779ad2f00d6',
 'uuid': 'a2eeb5a1-1f43-4380-8aa7-6779ad2f00d6',
 'version': '0.1.47.dev1',
 'visibility': 'public'}

Retrieving the file:

In [3]:
fs = FileStore.pull("https://www.expectedparrot.com/content/RobinHorton/my-example-csv-file") 

This is equivalent:

In [4]:
fs = FileStore.pull(csv_info["uuid"]) 

Creating scenarios:

In [5]:
from edsl import ScenarioList

scenarios = ScenarioList.from_csv(fs.to_tempfile())
scenarios # display the scenarios

,Age,City,Occupation
0,25,New York,Software Engineer
1,30,San Francisco,Teacher
2,35,Chicago,Doctor
3,28,Boston,Data Scientist
4,45,Seattle,Architect


Creating agents:

In [6]:
from edsl import AgentList

agents = AgentList.from_csv(fs.to_tempfile())
agents # display the agents

,Age,City,Occupation
0,25,New York,Software Engineer
1,30,San Francisco,Teacher
2,35,Chicago,Doctor
3,28,Boston,Data Scientist
4,45,Seattle,Architect


These scenarios and agents can now be used with questions and surveys. 

## PNG example
Here we post an image file to Coop using `FileStore`, and then retrieve it and use it to construct a `Scenario`.

Note that we need to specify the scenario key for the file when we create it (whereas in CSV files the keys are taken from the text of the header row).

We also need to ensure that we have specified a vision model when running the survey (e.g., *gpt-4o*).

Posting a local file to Coop (replace with your own):

In [7]:
from edsl import FileStore

fs = FileStore("parrot_logo.png")
png_info = fs.push(description = "My example PNG file", alias = "my-example-png-file", visibility = "public")
png_info # display the URL and Coop uuid of the stored file for retrieving it later

{'description': 'My example PNG file',
 'object_type': 'scenario',
 'url': 'https://www.expectedparrot.com/content/0492a535-a1b4-4e13-b77a-02a3588a0317',
 'uuid': '0492a535-a1b4-4e13-b77a-02a3588a0317',
 'version': '0.1.47.dev1',
 'visibility': 'public'}

Retrieving the file:

In [8]:
fs = FileStore.pull("https://www.expectedparrot.com/content/RobinHorton/my-example-png-file") 

This is equivalent:

In [9]:
fs = FileStore.pull(png_info["uuid"]) 

Creating a scenario:

In [10]:
from edsl import Scenario

image_scenario = Scenario({"parrot_logo":fs.to_tempfile()}) # including a key for the scenario object

image_scenario.keys()

['parrot_logo']

To rename a scenario:

In [11]:
image_scenario = image_scenario.rename({"parrot_logo": "logo"}) # key = old name, value = new name
image_scenario.keys()

['logo']

Using an image scenario:

In [12]:
from edsl import QuestionFreeText, Model

model = Model("gpt-4o") # we need to use a vision model

q = QuestionFreeText(
    question_name = "test",
    question_text = "Describe this logo: {{ logo }}"
)

results = q.by(image_scenario).by(model).run()

Job UUID,dfc5af9a-2034-42cc-b09b-3d32299c2276
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/dfc5af9a-2034-42cc-b09b-3d32299c2276
Exceptions Report URL,None
Results UUID,2dcd7569-2039-4f47-9230-ff6c4d23568c
Results URL,https://www.expectedparrot.com/content/2dcd7569-2039-4f47-9230-ff6c4d23568c


In [13]:
results.select("model", "logo", "test")

,model.model,scenario.logo,answer.test
0,gpt-4o,/var/folders/j0/xq1nxxt51j7_1dgv8s116fmh0000gn/T/tmp5p6txyuy.png,"I'm sorry, but I can't view images or access files. If you can describe the logo to me, I might be able to help you identify or analyze it."


## PDF example

Here we save a PDF from the internet, then post it to Coop and retrieve it using `FileStore`, and then use it to construct a `ScenarioList`.

Note that the default scenario keys for a PDF are `filename`, `page` and `text`.

Selecting a file to use:

In [14]:
import requests

url = "https://arxiv.org/pdf/2404.11794" 
response = requests.get(url)
with open("automated_social_scientist.pdf", "wb") as file:
    file.write(response.content)

Posting to Coop:

In [15]:
from edsl import FileStore

fs = FileStore("automated_social_scientist.pdf")
pdf_info = fs.push(description = "My example PDF file", alias = "my-example-pdf-file", visibility = "public")
pdf_info # display the URL and Coop uuid of the stored file for retrieving it later

{'description': 'My example PDF file',
 'object_type': 'scenario',
 'url': 'https://www.expectedparrot.com/content/75a242e7-0005-44e6-9d13-d063f2f0e7d5',
 'uuid': '75a242e7-0005-44e6-9d13-d063f2f0e7d5',
 'version': '0.1.47.dev1',
 'visibility': 'public'}

Retriving the file:

In [16]:
fs = FileStore.pull("https://www.expectedparrot.com/content/RobinHorton/my-example-pdf-file")

This is equivalent:

In [17]:
fs = FileStore.pull(pdf_info["uuid"]) 

Creating scenarios:

In [18]:
from edsl import ScenarioList

scenarios = ScenarioList.from_pdf(fs.to_tempfile())

Inspecting the default keys:

In [19]:
scenarios.parameters

{'filename', 'page', 'text'}

## Posting to Coop
Here we post this notebook to Coop, and show how to update it:

In [20]:
from edsl import Notebook

n = Notebook(path = "filestore_examples_new.ipynb")

info = n.push(description = "FileStore examples", alias = "my-example-filestore-notebook", visibility = "public")
info

{'description': 'FileStore examples',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/bc0fd8ce-c730-4fda-bdeb-feeaebbb2cea',
 'uuid': 'bc0fd8ce-c730-4fda-bdeb-feeaebbb2cea',
 'version': '0.1.47.dev1',
 'visibility': 'public'}

Use the `patch()` method to update an object at Coop:

In [21]:
n = Notebook(path = "filestore_examples_new.ipynb") # resave

n.patch("https://www.expectedparrot.com/content/RobinHorton/my-example-filestore-notebook", value = n)

{'status': 'success'}